<a id="top"></a>
# Calculating the Hubble Constant with eBOSS 
***
## Learning Goals


By the end of this tutorial, you will:

- Understand how to use `astroquery.mast` to download SDSS eBOSS and HST data from the MAST archive.
- Plot a sample of galaxies from eBOSS with a line of best fit to determine Hubble's constant.
- Create a color image of Abell 2199 from HST data.

## Table of Contents 
* [Introduction](#introduction)
* [Imports](#imports)
* [Accessing eBOSS Data from MAST](#accessing-eboss-data-from-mast)
    * [Querying eBOSS Data](#querying-eboss-data)
    * [Sampling eBOSS for Redshifted Galaxies](#sampling-eboss-for-redshifted-galaxies)
    * [Downloading Products from eBOSS](#downloading-products-from-eboss)
    * [Making the Hubble-Lemaître Diagram](#making-the-hubble-lemaître-diagram)
    * [Plotting eBOSS Data](#plotting-eboss-data)
    * [Comparing Calculated H<sub>0</sub> to Other Models](#comparing-calculated-h0-to-other-models)
* [Utilizing HST Data from Mast](#utilizing-hst-data-from-mast)
    * [Creating an Image of Abell 2199 Using HST](#creating-an-image-of-abell-2199-using-hst)
    * [Plotting the HST Images](#plotting-the-hst-images)
    * [Colorizing the HST Image](#colorizing-the-hst-image-of-abell-2199)
* [End of Tutorial](#end-of-tutorial)
* [Additional Resources](#additional-resources)
* [Citations](#citations)
* [About this Notebook](#about-this-notebook)




## Introduction 
The Hubble-Lemaître law is one of the most prominent topics of debate in modern cosmology. In 1927, Georges Lemaître published a paper in a Belgian journal proposing that the Universe was expanding, using data from Vesto Slipher at Lowell Observatory. Around the same time, Edwin Hubble was independently studying the relationship between galaxy distances and their radial velocities at Mount Wilson Observatory. Although Hubble was originally credited with the discovery, in 2018 the International Astronomical Union officially renamed "Hubble’s Law" to the "Hubble–Lemaître Law" to acknowledge Lemaître’s earlier contribution  [[1]](https://pressbooks.bccampus.ca/astronomy1105/chapter/26-5-the-expanding-universe/).

The Big Bang occurred approximately 14 billion years ago, leading to the expansion of the Universe. We can estimate the speed of this expansion by examining the redshifts of a sample of galaxies. These redshifts can be translated into radial velocities, and then using observed spectral flux, we can the determine the distances to these galaxies. By plotting radial velocities versus distances, we can apply a linear fit model to the data. This allows us to approximate the Hubble constant from the slope as described by Hubble's Law [[2]](https://www.teachastronomy.com/textbook/The-Expanding-Universe/Relating-Redshift-and-Distance/):

$$
v_{rad} = H_{0}d
$$

There are various models used to calculate H<sub>0</sub>, some based on observations of the Cosmic Microwave Background (CMB), and others on measurements of galaxies or supernovae. These models typically yield varying values of H<sub>0</sub> ranging from approximately 67 km/s/Mpc to 74 km/s/Mpc. This discrepancy is known as the *Hubble Tension* [[3]](https://doi.org/10.3390/universe9020094). 

In this notebook, we'll construct a Hubble-Lemaître Diagram using data from the Sloan Digital Sky Survey’s (SDSS) Extended Baryon Oscillation Spectroscopic Survey (eBOSS). eBOSS gathered over 4 million spectra from stars, quasars, and galaxies. We'll focus on a subset of galaxies, using their redshifts and spectral data to plot radial velocity against distance. By applying a linear fit to this relationship, we can estimate Hubble's constant from the slope of the resulting line.

## Imports 
The main packages and their use-cases in this tutorial are as follows:

- *numpy* to handle array functions
- *matplotlib.pyplot* for plotting data 
- *astroquery.mast.Observations* to access data from MAST
- *astropy.io.fits* for accessing FITS files
- *astropy.table.vstack* for vertically stacking tables
- *scipy.optimize.curve_fit* for creating a linear fit 
- *astropy.visualization.make_luptopn_rgb* for creating an RGB image 
- *astropy.wcs.WCS* for world coordinate system transformations
- *reproject* for stacking FITS images

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from astroquery.mast import Observations
import astropy.io.fits as fits
from astropy.table import vstack
from scipy.optimize import curve_fit

from astropy.visualization import make_lupton_rgb
from astropy.wcs import WCS
import reproject 

If you're not sure if you have the required versions of packages installed on your device, you can run the following cell:

In [ ]:
with open('requirements.txt') as f:
    print(f"Required packages for this notebook:\n{f.read()}")

To ensure these requirements are installed, you can run the following command in the terminal:
```bash
pip install -r requirements.txt
```

***

## Accessing eBOSS Data from MAST 

The [Mikulski Archive for Space Telescopes (MAST)](https://archive.stsci.edu/) hosts a large array of data from several telescope missions. In this tutorial, we will be specifically focusing on data from the Sloan Digital Sky Survey (SDSS) and the Hubble Space Telescope (HST). 

The [Extended Baryon Oscillation Spectroscopic Survey (eBOSS)](https://www.sdss4.org/surveys/eboss/) was completed using the SDSS-2.5m telescope and BOSS spectrograph at the Apache Point Observatory in New Mexico. Using plug plates and a 1000 fiber-fed spectrograph, eBOSS was able to collect spectra from over 4 million objects including quasars, galaxies, and stars in the northern hemisphere.

### Querying eBOSS Data 

You can query data from MAST using the website portal [here](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html). You can also query MAST data in Python using the package `astroquery.mast`! 


We can query all of the eBOSS data using `Observations.query_criteria` with `provenance_name = 'eBOSS'`. Since eBOSS has a large amount of data, we can display the first 10 results using the `pagesize` parameter as well as the `page` parameter. 

This might take a few minutes depending on your internet connection.

In [ ]:
eboss_data = Observations.query_criteria(provenance_name='eBOSS', pagesize=10, page=1)

# Show the astropy table
eboss_data

The table above provides some basic information for each object:
- `s_ra` and `s_dec`: Right ascension and declination.
- `instrument_name`: `BOSS` indicates that the data were collected using the BOSS spectrograph.
- `obs_id`: Observation ID associated with the object. This is based on the plate number, the modified Julian date of observation, and the fiber ID, so the observation ID will look something like `sdss_eboss_{PLATE}-{MJD}-{FIBERID}`.
- `target_classification`: Indicates the type of object (`QSO`, `STAR`, or `GALAXY`).
- `t_min` and `t_max`: The modified Julian dates indicating when the object was first and last observed.
- `wavelength_region`: Indicates the region of the electromagnetic spectrum observed. This should be `OPTICAL`, since eBOSS observed in the optical wavelength range.
- `em_min` and `em_max`: The minimum and maximum wavelengths observed by the survey. For eBOSS, this range is 360.0–1040.0 nanometers (optical).
- `dataproduct_type`: Since eBOSS was a spectroscopic survey, this should be `spectrum`.

### Sampling eBOSS for Redshifted Galaxies 

In this tutorial, we will be examining a sample of 311 galaxies, with 32 of those being from the galaxy cluster Abell 2199. This sample was curated for this exercise by randomly selecting 500 galaxies from the LOFAR subprogram of eBOSS across a uniform range of redshifts, and then selecting the most linear portion of the data when plotted on a radial velocity vs. distance plot. 

For the purpose of this tutorial, we will only focus on querying by observation ID. The list of observation IDs of the 311 galaxies is located in the .txt file `all_galaxies.txt`. The galaxies from *Abell 2199* are located in `abell_2199.txt`. 

In [ ]:
# importing the galaxy IDs
obs_id = np.loadtxt('all_galaxies.txt', dtype=str)

# looping through the .txt file to collect query results (this might take a few minutes)
results_tables = []
for i in obs_id:
    query_result = Observations.query_criteria(provenance_name='eBOSS', obs_id=i)
    results_tables.append(query_result)

# combine query results using vstack
combined_results = vstack(results_tables)

In [ ]:
# show the first 10 results
combined_results[:10]

### Downloading Products from eBOSS

For each object, there are three available files: The preview .png image, the "full spectrum" file, and the "lite spectrum" file. The full spectrum is listed as `Minimum Recommended Products` for the `productGroupDescription` parameter. 

For more information about eBOSS data products, you can visit the MAST Archive Manual [here](https://outerspace.stsci.edu/display/SDSS/eBOSS+Data+Products) as well as in the search results table. 

This will take a few minutes to load.

In [ ]:
# Obtaining the product list of all galaxies
products = Observations.get_product_list(combined_results)

# Display the available products
products

To utilize the data, we must download the FITS files. Any data products dowloaded from MAST will appear in a folder called `mastDownload`. 

We'll be focusing on using the full spectrum datafile only, which is labeled "Minimum Recommended Products" as the product Group Description. To ensure we're only downloading this file for each target, we'll use the parameter `mrp_only = True`. 

This download should take around 45 seconds or so. 

In [ ]:
# Downloading the full spectra products
manifest = Observations.download_products(products, mrp_only=True)

We'll send these FITS files to an array so that we can access the redshift and flux values.

In [ ]:
# Sending the FITS files to an array
galaxy_array = []

for path in manifest["Local Path"]:
    galaxy_array.append(fits.open(path))

In [ ]:
# View available file extensions 
galaxy_array[0].info()

For this tutorial, we will focus on using the spectroscopic flux in the r-band provided by spALL. These are accessed through the second file extension. 

The various file extensions are broken down [here](https://data.sdss.org/datamodel/files/BOSS_SPECTRO_REDUX/RUN2D/spectra/PLATE4/spec.html):
- HDU 0: The primary header information. 
- HDU 1: The coadded observed spectrum.
- HDU 2: Metadata from spALL. 
- HDU 3: Emission line fitting metadata from spZline.
- HDU 4+: Individual red and blue spectra that make up the coadded spectrum. 

The second file extension is broken down even further [here](https://data.sdss.org/datamodel/files/BOSS_SPECTRO_REDUX/RUN2D/PLATE4/RUN1D/spZbest.html). We'll be focusing on using `SPECTROFLUX`. Since this provides us a five-element array for each band (u, g, r, i , z), we'll use the third element for the r-band flux. 


### Making the Hubble-Lemaître Diagram

In [ ]:
# Setting up arrays to send all z-values, z-error, and flux to arrays
z_val = []
z_err = []
r_flux = []

# Looping through all the data and adding z-values, z-error, and flux to arrays
for galaxy in galaxy_array:
    galaxy_z = galaxy[2].data["Z"]
    zerr = galaxy[2].data["Z_ERR"]
    spectroflux = galaxy[2].data["SPECTROFLUX"][:, 2]
    z_val.append(galaxy_z)
    z_err.append(zerr)
    r_flux.append(spectroflux)

# Converting lists to arrays 
z_val = np.array(z_val)
z_err = np.array(z_err)
r_flux = np.array(r_flux)

In order to determine the relative distance to each galaxy, we can use the inverse square law as we know flux is proportional to $\frac{1}{d^2}$. Thus, distance is proportional to $\frac{1}{\sqrt{flux}}$ [[4]](https://skyserver.sdss.org/dr12/en/proj/advanced/hubble/distances.aspx). 

In [ ]:
# Finding the relative distance by taking 1/sqrt of the radiant flux
relative_distance = 1/np.sqrt(r_flux)

# Converting the relative distance to kiloparsecs
new_relative_distance = relative_distance/1e3 # kpc 

# Stacking arrays to remove points with nan
plot_array = np.column_stack((z_val, new_relative_distance, z_err))

Due to the behavior of $\frac{1}{\sqrt{flux}}$, we will create a mask to remove any 'nan' values.

In [ ]:
# Selecting all distances that are not 'nan' 
mask = ~np.isnan(plot_array[:, 1]) 

# Applying mask
new_plot_array = plot_array[mask]

Since Hubble's constant is in units km/s/Mpc, we want to convert redshift to velocity. To do this, we can use the relativistic radial velocity equation [[5]](https://ned.ipac.caltech.edu/level5/Hogg/Hogg3.html): <br>
$$
v = c \frac{(1+z)^2-1}{(1+z)^2+1}
$$

Using error propagation, the error in redshift becomes: <br>
$$
v_{err} = z_{err}|\frac{4c(1+z)}{((1+z)^2+1)^2}|
$$

In [ ]:
# Define speed of light constant (c) in km/s
c = 2.998e5 # km/s

# Calculating velocity from redshift
velocity = []
velocity_err = []
for i, j in zip(new_plot_array[:, 0], new_plot_array[:, 2]):
    v = c * (((1 + i)**2 - 1) / ((1 + i)**2 + 1))
    v_err = np.abs(((4 * c * (1 + i)) / ((1 + i)**2 + 1)**2)) * j
    velocity.append(v)
    velocity_err.append(v_err)

# Creating new array of distance, velocity, and error in velocity
distance_velocity_array = np.column_stack((new_plot_array[:, 1], velocity, velocity_err))

### Plotting eBOSS Data

We can examine the data by creating a scatter plot of the velocity versus the relative distance.

In [ ]:
# Plotting eBOSS data
plt.figure(figsize=(8, 5))
plt.scatter(distance_velocity_array[:, 0], distance_velocity_array[:, 1], s=1.5, color='blue')
plt.title('Velocity vs. Relative Distance of eBOSS Data')
plt.xlabel('Relative Distance')
plt.ylabel('Radial Velocity (km/s)')
plt.grid(True)
plt.show()

In order to obtain the actualized distances for the sample, we can use the relative distance of the first galaxy from Abell 2199 and set it equal to 1. We'll follow the same procedures as before by importing the `abell_2199.txt` file and querying eBOSS for those 32 select galaxies.

In [ ]:
# Loading the Abell 2199 dataset
obs_id2 = np.loadtxt('abell_2199.txt', dtype=str)

# Querying eBOSS for the 32 Abell 2199 galaxies, sending the data to an array 
results_tables2 = []
for i in obs_id2:
    query_result2 = Observations.query_criteria(provenance_name='eBOSS', obs_id=i)
    results_tables2.append(query_result2)

# Stacking the data into one table 
combined_results2 = vstack(results_tables2)

# Ensuring the length of the stacked table is 32
print(f"Length of Abell 2199 dataset = {len(combined_results2)}")

We'll gather and download the products again, and then send the FITS data to an array.

In [ ]:
# Gathering and downloading full spectra files
products2 = Observations.get_product_list(combined_results2)
manifest2 = Observations.download_products(products2, mrp_only=True)

# Writing data to an array 
abell_array = []
for path in manifest2["Local Path"]:
    abell_array.append(fits.open(path))

# Checking the length of the array matches the dataset 
print(f"Length of Abell 2199 array = {len(abell_array)}")

Now we can extract the redshift, error in redshift, and r-band flux values. Following the same processes as before, we can use the flux to find the relative distances.

In [ ]:
# Sending redshift, error in redshift, and r-band magnitudes to arrays
z_val_abell = []
z_err_abell = []
r_flux_abell = []

for galaxy in abell_array:
    galaxy_z = galaxy[2].data["Z"]
    zerr = galaxy[2].data["Z_ERR"]
    spectroflux = galaxy[2].data["SPECTROFLUX"][:, 2]
    z_val_abell.append(galaxy_z)
    z_err_abell.append(zerr)
    r_flux_abell.append(spectroflux)

# Converting lists into arrays
z_val_abell = np.array(z_val_abell)
z_err_abell = np.array(z_err_abell)
r_flux_abell = np.array(r_flux_abell)

# Calculate relative distance 
relative_distance_abell = 1/np.sqrt(r_flux_abell)
new_relative_distance_abell = relative_distance_abell/1e3 # kpc 

# Filter out zeros
plot_array_abell = np.column_stack((z_val_abell, new_relative_distance_abell, z_err_abell))
mask2 = ~np.isnan(plot_array_abell[:, 1])
new_array_abell = plot_array_abell[mask2]

# Calculate velocity
velocity_abell = []
velocity_err_abell = []
for i, j in zip(new_array_abell[:, 0], new_array_abell[:, 2]):
    v = c * (((1 + i)**2 - 1) / ((1 + i)**2 + 1))
    v_err = np.abs(((4 * c * (1 + i)) / ((1 + i)**2 + 1)**2)) * j
    velocity_abell.append(v)
    velocity_err_abell.append(v_err)

# Creating new array of distance, velocity, and error in velocity
distance_velocity_array_abell = np.column_stack((new_array_abell[:, 1], velocity_abell, velocity_err_abell))

Now that we have filtered out both datasets, we can normalize the relative distances. To do this, we'll set up a ratio such that the closest galaxy in the Abell 2199 cluster equals 1:
$$
\frac{d_{1}}{d_{relative}} = \frac{1}{d_{norm}}
$$
where $d_{1}$ is the relative distance of the closest Abell 2199 galaxy, $d_{relative}$ is the array of relative distances of the eBOSS galaxies, and $d_{norm}$ is the array of normalized distances. To solve for the array of $d_{norm}$, we'll divide the array of relative distances by $d_{1}$. [[4]](https://skyserver.sdss.org/dr12/en/proj/advanced/hubble/distances.aspx)

Since we know the distance to Abell 2199 is 128 Mpc, we'll multiply the normalized distances by 128 Mpc to obtain the actualized distances. [[6]](http://ned.ipac.caltech.edu/cgi-bin/nph-objsearch?objname=Abell+2199&extend=no&hconst=73&omegam=0.27&omegav=0.73&corr_z=1&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES).


In [ ]:
# Sorting the Abell 2199 data to find closest galaxy
sorted_abell = np.argsort(distance_velocity_array_abell[:, 0])
sorted_abell_array = distance_velocity_array_abell[sorted_abell]

# We'll take the first Abell 2199 galaxy and find the average, then set that to 1
d1_abell = sorted_abell_array[0][0]

# Normalize distances to d1_abell 
norm_dist = distance_velocity_array[:, 0]/d1_abell
norm_dist_abell = distance_velocity_array_abell[:, 0]/d1_abell

# Distance to Abell 2199 
distance = 128 # Mpc

Now we have normalized distances! We can multiply these by the distance to Abell 2199 and generate a plot with distances and radial velocities. We can use `scipy.optimize.curve_fit` to perform a linear fit, and in return, we can obtain the value of the slope, i.e., Hubble's constant!

In [ ]:
# Defining linear fit function
def fit_func(x, m, b):
    return m * x + b 

In [ ]:
# Doing linear fit using scipy.optimize.curve_fit
popt, covariance = curve_fit(fit_func, norm_dist * distance, distance_velocity_array[:, 1], sigma=distance_velocity_array[:, 2], absolute_sigma=True)
m = popt[0] # slope (Hubble's constant!)
b = popt[1] # intercept

# Setting up an array of x-values to plot the line of best fit
x_array = np.linspace(0, 2000, 200000)

# Plotting the data as well as the line of best fit
plt.errorbar(norm_dist * distance, distance_velocity_array[:, 1], yerr=distance_velocity_array[:, 2], ms=1.5, fmt='o', label='eBOSS data', zorder=1)
plt.scatter(norm_dist_abell * distance, distance_velocity_array_abell[:, 1], s=5, label='Abell 2199', color='purple', zorder=2)
plt.plot(x_array, fit_func(x_array, m, b), label='linear fit', color='orange', zorder=3)
plt.ylabel('Radial Velocity (km/s)')
plt.xlabel('Actualized Distance (Mpc)')
plt.title(f"Radial Velocity vs. Actualized Distance of eBOSS Galaxy Sample \n eBOSS - LOFAR \n $H_{0}$ = {m:.3f} km/s/Mpc \n r - band magnitude")
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.show()

### Comparing Calculated H<sub>0</sub> to Other Models 

Using data from eBOSS, we were able to calculate an H<sub>0</sub> value of $\approx$ 73.66 km/s/Mpc! 

The European Space Agency calculated an H<sub>0</sub> value of 67.40 ± 1.40 km/s/Mpc using Planck data. A recent calculation using Cepheid variables and SNe Ia from the SH0ES collaboration provided an H<sub>0</sub> value of 73.04 ± 1.04 km/s/Mpc [[3]](https://doi.org/10.3390/universe9020094). Astronomers continue to use both local distance indicators (e.g., Tip of the Red Giant Branch stars) and early-Universe probes like the CMB to constrain H<sub>0</sub>, but the Hubble Tension remains unresolved [[7]](https://www.scientificamerican.com/article/the-hubble-tension-is-becoming-a-hubble-crisis/).

Due to relativistic effects, H<sub>0</sub> values calculated from CMB observations (which probe the early Universe) tend to be lower than those calculated from nearby objects like Cepheid variables or Type Ia supernovae. Since our H<sub>0</sub> value is based on galaxy observations from eBOSS, it's reasonable that it falls on the higher end of the range, though this could also be influenced by uncertainties introduced by several assumptions:
- We assume galaxies are point sources, using a single static distance for each. 
- We calculate distances using the inverse square law, but intrinsic galaxy brightness can vary significantly. 

***
## Utilizing HST Data from MAST

The Hubble Space Telescope (HST) is a 2.4 meter serviceable telescope launched in 1990. It's equipped with three cameras (ACS, WFC3/UVIS, and WFC3/IR). Using observations from HST, we can construct an image of Abell 2199!

While we won't be covering this in the tutorial, we could use the constructed image from HST to determine the relative distance to Abell 2199 rather than using the one from NED. For example, we could use the apparent brightness of the cluster’s brightest galaxy, or analyze the relative physical sizes of its member galaxies [[4]](https://skyserver.sdss.org/dr12/en/proj/advanced/hubble/distances.aspx).

For more information about HST, check out this [overview from MAST](https://archive.stsci.edu/missions-and-data/hst)!

### Creating an Image of Abell 2199 Using HST 

To create an image of Abell 2199 using HST, we'll query MAST using `objectname = 'Abell 2199'` and specify HST using `obs_collection = 'HST'`. 

In [ ]:
# Query HST using objectname
obs_hst = Observations.query_criteria(objectname='Abell 2199', 
                                      # specifying Hubble data
                                      obs_collection='HST',
                                      # We want science observations and not calibration files
                                      intentType='science',
                                      # Calibrated reduced observations to display final image
                                      provenance_name='CAL*')
# Display results 
obs_hst

Similar to eBOSS, we'll retrieve and download the products for Abell 2199, specifically the fully calibrated combined images using `productGroupDescription = 'Minimum Recommended Products'` and `productSubGroupDescription = 'DRZ'`. 

In [ ]:
# Get products from query 
products_hst = Observations.get_product_list(obs_hst)

# Select specific products
products_hst = Observations.filter_products(products_hst,
                                            # Specify science files
                                            productType='SCIENCE',
                                            # Recommended products,
                                            productGroupDescription='Minimum Recommended Products',
                                            # DRZ files -> calibrated combined images
                                            productSubGroupDescription='DRZ')
# Display product list
products_hst

In [ ]:
# Download products into same directory
Observations.download_products(products_hst, flat=True)

Each FITS file has a corresponding filter in the table above under the `filter` column. To create a color image, we'll choose three images that overlap coordinates and have different filters. 

The following images were picked based on if they overlapped in the AstroView of the [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html).

In [ ]:
# Opening three different filters 
img1 = fits.open('j6jt05011_drz.fits')
img1_wcs = WCS(img1[1].header)

img2 = fits.open('j6jt05021_drz.fits')
img2_wcs = WCS(img2[1].header)

img3 = fits.open('iey929020_drz.fits')
img3_wcs = WCS(img3[1].header)

#### Plotting the HST Images 

We'll now set up a subplot of the three images we chose. The first two images were taken as a pair of images (hence the small whitespace in the center). You can see the overlap in all three images where there's a fairly consistent cluster of five galaxy sources. While the shapes of these galaxies aren't very clear in these images, they'll become more distinct when we create the color image!

In [ ]:
# Set up figure
fig = plt.figure(figsize=(15, 8))

# First image
ax1 = fig.add_subplot(1, 3, 1, projection=img1_wcs)
ax1.imshow(img1[1].data, cmap='Greys_r', vmin=0, vmax=2.5, origin='lower')
ax1.set_title(f"{img1[0].header['INSTRUME']}/{img1[0].header['FILTER1']}")
ax1.set_ylabel('Declination')
ax1.set_xlabel('Right Ascension')

# Second image
ax2 = fig.add_subplot(1, 3, 2, projection=img2_wcs)
ax2.imshow(img2[1].data, cmap='Greys_r', vmin=0, vmax=2.5, origin='lower')
ax2.set_title(f"{img2[0].header['INSTRUME']}/{img2[0].header['FILTER1']}")
ax2.set_ylabel('Declination')
ax2.set_xlabel('Right Ascension')

# Third image
ax3 = fig.add_subplot(1, 3, 3, projection=img3_wcs)
ax3.imshow(img3[1].data, cmap='Greys_r', vmin=0, vmax=2.5, origin='lower')
ax3.set_title(f"{img3[0].header['INSTRUME']}/{img3[0].header['FILTER']}")
ax3.set_ylabel('Declination')
ax3.set_xlabel('Right Ascension')

plt.tight_layout()
plt.show()

#### Colorizing the HST Image of Abell 2199 

In order to create a color image of Abell 2199, we can assign red, green, and blue to each image, then combine the three images. We must choose the image with the largest pixel dimensions, and then project the other images onto it.  

In [ ]:
# Checking the shape of each image to determine which ones will be mapped onto the other
print(f"Shape of image 1: {img1[1].data.shape}")
print(f"Shape of image 2: {img2[1].data.shape}")
print(f"Shape of image 3: {img3[1].data.shape}")

The third image appears to have the largest dimensions, so we'll project the first and second images onto the third using `reproject`. We'll assign the first image to red, the second to blue, and the third to green. 

In [ ]:
# image 1 -> red 
r, _ = reproject.reproject_interp(img1[1], img3[1].header)

# image 2 -> green
g, _ = reproject.reproject_interp(img2[1], img3[1].header)

# image 3 -> blue
b, _ = reproject.reproject_interp(img3[1], img3[1].header)

Using a ratio of red, blue, and green, we can now make a colorized image using `make_lupton_rgb`!

In [ ]:
image_hst = make_lupton_rgb(r * 3.8, g * 0.2, b * 0.65, Q=4, stretch=0.75)

In [ ]:
# Plotting the colorized image 
plt.figure(figsize=(5, 5))

# Ensuring the coordinates are set to the WCS of the third image
ax = plt.subplot(projection=img3_wcs)
ax.imshow(image_hst)

# Zoom in
ax.set_xlim(1400, 2100)
ax.set_ylim(2200, 2900)

ax.set_xlabel('Right Ascension')
ax.set_ylabel('Declination')
ax.set_title('Colorized Image of Abell 2199')
ax.legend()
plt.tight_layout()
plt.show()

***
## End of Tutorial 
Congratulations, you've reached the end of this notebook! You've learned how to use `astroquery.MAST` to calculate Hubble's constant from eBOSS data and make a colorized image from HST data! 

## Additional Resources 

Additional resources are linked below:

- [SDSS Legacy Archive at MAST](https://archive.stsci.edu/missions-and-data/sdss)
- [SDSS Legacy Archive at MAST User Manual](https://outerspace.stsci.edu/display/SDSS/The+SDSS+Legacy+Archive+at+MAST)
- [eBOSS User Manual](https://outerspace.stsci.edu/display/SDSS/eBOSS)
- [astroquery.mast User Manual](https://astroquery.readthedocs.io/en/latest/mast/mast.html)
- [MAST API](https://mast.stsci.edu/api/v0/index.html)

## Citations 

If you use data from MAST for published research, please see the following links for information on which citations to include in your paper:

* [Citing SDSS](https://sdss.org/collaboration/citing-sdss/)
* [Citing MAST](https://archive.stsci.edu/publishing/mission-acknowledgements)
* [Citing astropy](https://www.astropy.org/acknowledging.html)

## About this Notebook

**Author(s):** Julie Imig (jimig@stsci.edu), Natalie Haugen (nhaugen@terpmail.umd.edu) <br>
**Keyword(s):** Tutorial, eBOSS, HST, SDSS, galaxies <br>
**First published:** July 2025 <br>
**Last updated:** July 2025 <br>

***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/style-guides/master/guides/images/stsci-logo.png" alt="Space Telescope Logo" width="200px"/> 